# HydroHomies Plots
In this notebook, the plots, figures and also some explanations or details about each of them are being presented.  

To clarify, plaase fallow this order:
- Title for each plot is mandatory
- Analysis must be written 
- Address of data should be mentioned

Loading in all our data

In [25]:
import yaml
import pandas as pd
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)

# Data blue    
personal_data_blue_df = pd.read_excel(config['personal_data_blue'], sheet_name = None)
verbal_fluency_blue_df = pd.read_excel(config['verbal_fluency_test_blue'], sheet_name = None)
stroop_test_blue_df = pd.read_excel(config['stroop_test_blue'], sheet_name = None)
stop_signal_test_blue_df = pd.read_excel(config['stop_signal_test_blue'], sheet_name = None)
flanker_test_blue_df = pd.read_excel(config['flanker_test_blue'], sheet_name = None)
digit_span_test_blue_df = pd.read_excel(config['digit_span_test_blue'], sheet_name = None)

# Data green
personal_data_green_df = pd.read_excel(config['personal_data_green'], sheet_name = None)
verbal_fluency_green_df = pd.read_excel(config['verbal_fluency_test_green'], sheet_name = None)
stroop_test_green_df = pd.read_excel(config['stroop_test_green'], sheet_name = None)
stop_signal_test_green_df = pd.read_excel(config['stop_signal_test_green'], sheet_name = None)
flanker_test_green_df = pd.read_excel(config['flanker_test_green'], sheet_name = None)
digit_span_test_green_df = pd.read_excel(config['digit_span_test_green'], sheet_name = None)

#Data red
personal_data_red_df = pd.read_excel(config['personal_data_red'], sheet_name = None)
verbal_fluency_red_df = pd.read_excel(config['verbal_fluency_test_red'], sheet_name = None)
stroop_test_red_df = pd.read_excel(config['stroop_test_red'], sheet_name = None)
stop_signal_test_red_df = pd.read_excel(config['stop_signal_test_red'], sheet_name = None)
flanker_test_red_df = pd.read_excel(config['flanker_test_red'], sheet_name = None)
digit_span_test_red_df = pd.read_excel(config['digit_span_test_red'], sheet_name = None)

# Data purple
personal_data_purple_df = pd.read_excel(config['personal_data_purple'], sheet_name = None)
verbal_fluency_purple_df = pd.read_excel(config['verbal_fluency_test_purple'], sheet_name = None)
stroop_test_purple_df = pd.read_excel(config['stroop_test_purple'], sheet_name = None)
stop_signal_test_purple_df = pd.read_excel(config['stop_signal_test_purple'], sheet_name = None)
flanker_test_purple_df = pd.read_excel(config['flanker_test_purple'], sheet_name = None)
digit_span_test_purple_df = pd.read_excel(config['digit_span_test_purple'], sheet_name = None)

# Data pink
personal_data_pink_df = pd.read_excel(config['personal_data_pink'], sheet_name = None)
verbal_fluency_pink_df = pd.read_excel(config['verbal_fluency_test_pink'], sheet_name = None)
stroop_test_pink_df = pd.read_excel(config['stroop_test_pink'], sheet_name = None)
stop_signal_test_pink_df = pd.read_excel(config['stop_signal_test_pink'], sheet_name = None)
flanker_test_pink_df = pd.read_excel(config['flanker_test_pink'], sheet_name = None)
digit_span_test_pink_df = pd.read_excel(config['digit_span_test_pink'], sheet_name = None)

# Data orange
personal_data_orange_df = pd.read_excel(config['personal_data_orange'], sheet_name = None)
verbal_fluency_orange_df = pd.read_excel(config['verbal_fluency_test_orange'], sheet_name = None)
stroop_test_orange_df = pd.read_excel(config['stroop_test_orange'], sheet_name = None)
stop_signal_test_orange_df = pd.read_excel(config['stop_signal_test_orange'], sheet_name = None)
flanker_test_orange_df = pd.read_excel(config['flanker_test_orange'], sheet_name = None)
digit_span_test_orange_df = pd.read_excel(config['digit_span_test_orange'], sheet_name = None)

# Figures



### **Comparison of Dehydration and control of one person**

### Explanations: 

### Data: 

In [6]:
# the code
pass

### Analysis: 

---

### **Comparison of Dehydration of all participants**

### Explanations: 

#### Data: 

In [7]:
#code

### Analysis: 

---

### **Comparison of Rehydration of all participants**

### Explanations: 

### Data: 

In [8]:
#code

### Analysis: 

---